#  用CNN(卷积神经网络)识别新闻数据中的命名实体

*在这个教程中，我们会用卷积神经网络（convolutional neural network,CNN）去解决命名实体识别（Named Entity Recognition,NER）的问题。*
  
*命名实体识别是自然语言处理中经常遇到的问题，它的作用是从文本中抽取出一些实体，例如人、机构、地点等等。* 
  
*在这里，我们会做些实验，在CoNLL-2003数据集中的不同新闻中，识别出命名实体。*  
  
  例如，我们想从下面这句话中解析出人和机构的名字
>Yan Goodfellow works for Google Brain
  
  NER模型需要提供如下的标签(tags)序列:
>B-PER I-PER    O     O   B-ORG  I-ORG
  
  
  这里有两个前缀：  
*B-*代表着实体的beginning  
*I-*代表着实体的inside  
*O*代表没有标签  
带有这种前缀的标记称为BIO标记(BIO markup),引入此标记是为了区分具有相似类型的后续实体。  
  
  
  解决这种问题需要用到神经网络的相关知识，尤其是**卷积神经网络**。  
  


## 数据
下面的单元格将把这个任务所需的所有数据下载到文件夹 \data 中，库中的下载工具用来下载和提取文件的。

In [1]:
import deeppavlov
from deeppavlov.core.data.utils import download_decompress
download_decompress('http://files.deeppavlov.ai/deeppavlov_data/conll2003_v2.tar.gz', 'data/')

2018-08-19 16:51:27.649 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 205: Starting new HTTP connection (1): files.deeppavlov.ai:80
2018-08-19 16:51:28.650 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 393: http://files.deeppavlov.ai:80 "GET /deeppavlov_data/conll2003_v2.tar.gz HTTP/1.1" 302 None
2018-08-19 16:51:28.659 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 205: Starting new HTTP connection (1): 202.112.144.234:80
2018-08-19 16:51:28.671 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 393: http://202.112.144.234:80 "GET /files/3146000001CF3D15/lnsigo.mipt.ru/export/deeppavlov_data/conll2003_v2.tar.gz HTTP/1.1" 200 957092
2018-08-19 16:51:28.675 INFO in 'deeppavlov.core.data.utils'['utils'] at line 62: Downloading from http://files.deeppavlov.ai/deeppavlov_data/conll2003_v2.tar.gz to data/conll2003_v2.tar.gz
100%|██████████| 957k/957k [00:00<00:00, 1.30MB/s]
2018-08-19 16:51:29.419 INFO in 'deeppavlov.core.data.utils'['utils'

## 加载CoNLL-2003命名实体识别语料库
这里我们将运用到一个包含带有命名实体标签的推特文章的语料库(corpus)。一个典型的命名实体识别数据文件包含符号（*tokens*）（词或标点符号）和标签（*tags*），它们被空格分隔开。有的时候一些附加信息，比如 POS-tags 也是包含在其中的。
不同的文件是用 **-DOCSTART-** 开头的一行分隔开的，不同的句子是用一行空白行分隔开的。
例如：
~~~
  
  -DOCSTART- -X- -X- O

  EU NNP B-NP B-ORG  
  rejects VBZ B-VP O  
  German JJ B-NP B-MISC  
  call NN I-NP O  
  to TO B-VP O  
  boycott VB I-VP O  
  British JJ B-NP B-MISC  
  lamb NN I-NP O  
  . . O O  

  Peter NNP B-NP B-PER  
  Blackburn NNP I-NP I-PER  
  
~~~
这个教程中我们只关注tokens和tags（也就是每行的第一个元素和最后一个元素）,而忽略掉两者之间的POS元素。  
  

我们先新建一个Conll2003DatasetReader类，用来读取数据集。它返回的是一个dictionary包含train,test,valid这三个field，每个field存储着一些sample构成的list，每个sample是由tokens和tags构成的tuple,其中tokens和tags是list。
下面的例子描述了这个dictionary的结构，它由NerDatasetReader类中的read()方法返回：  
~~~
{'train': [(['Mr.', 'Dwag', 'are', 'derping', 'around'], ['B-PER', 'I-PER', 'O', 'O', 'O']), ....],
 'valid': [...],
 'test': [...]}

~~~  
数据集分为三个部分：  
1.train: 用来训练模型  
2.valid: 用来评估以及参数调优  
3.test:用来最终评估模型  
这三个部分分别存在三个txt文件中。  
  
我们会用库中的Conll2003DatasetReader类来读取数据，也就是把文本转换成如上所说的形式。


In [3]:
from deeppavlov.dataset_readers.conll2003_reader import Conll2003DatasetReader
dataset = Conll2003DatasetReader().read('data/')

我们应该始终了解我们处理的数据类型，因此，我们用下面的代码把它们打印出来。

In [12]:
for sample in dataset['train'][:5]:
    for token,tag in zip(*sample):
        print('%s\t%s' % (token,tag))
    print()

<DOCSTART>	O

EU	B-ORG
rejects	O
German	B-MISC
call	O
to	O
boycott	O
British	B-MISC
lamb	O
.	O

Peter	B-PER
Blackburn	I-PER

BRUSSELS	B-LOC
1996-08-22	O

The	O
European	B-ORG
Commission	I-ORG
said	O
on	O
Thursday	O
it	O
disagreed	O
with	O
German	B-MISC
advice	O
to	O
consumers	O
to	O
shun	O
British	B-MISC
lamb	O
until	O
scientists	O
determine	O
whether	O
mad	O
cow	O
disease	O
can	O
be	O
transmitted	O
to	O
sheep	O
.	O



## 准备字典
为了训练出来一个神经网络，我们需要用到两个映射（mapping）：  
* {token}$\to${token id}: 为当前的token处理嵌入矩阵中的行
* {tag}$\to${tag id}:制造one-hot地面真值概率分布向量，用于计算网络输出损耗。  
  
库中的 SimpleVocabulary 将会执行这些映射。

In [13]:
from deeppavlov.core.data.simple_vocab import SimpleVocabulary

接下来，我们将会为token和tag准备字典。有时词汇表中有一些特殊的token，例如一个未知的单词标记，每当我们遇到词汇表之外的单词时就会使用它。这种情况下，我们就会用< UNK > 这种特殊的记号来表示词汇表之外的单词。

In [14]:
special_tokens = ['<UNK>']
token_vocab = SimpleVocabulary(special_tokens, save_path='model/token.dict')
tag_vocab = SimpleVocabulary(save_path='model/tag.dict')

2018-08-19 18:18:25.903 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 53: No load path is set for SimpleVocabulary in 'infer' mode. Using save path instead
2018-08-19 18:18:25.904 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 53: No load path is set for SimpleVocabulary in 'infer' mode. Using save path instead


然后我们在数据的训练部分中加入词汇表。

In [17]:
all_tokens_by_sentences = [tokens for tokens, tags in dataset['train']]
all_tags_by_sentences = [tags for tokens, tags in dataset['train']]# 这是list

token_vocab.fit(all_tokens_by_sentences)
tag_vocab.fit(all_tags_by_sentences)

尝试得到索引，请记住，我们正在使用以下结构的批次:
~~~
[['utt0_tok0', 'utt1_tok1', ...], ['utt1_tok0', 'utt1_tok1', ...], ...]
~~~

In [18]:
token_vocab([['How', 'to', 'do', 'a', 'barrel', 'roll', '?']])

[[10167, 6, 168, 7, 6097, 5518, 1865]]

In [19]:
tag_vocab([['O', 'O', 'O'], ['B-ORG', 'I-ORG']])

[[0, 0, 0], [3, 5]]

In [22]:
tag_vocab([['I-ORG']])

[[5]]

接下来，我们试试从索引到token的转化。

In [23]:
import numpy as np
token_vocab([np.random.randint(0, 512, size=10)])

[['at',
  '2',
  'Germany',
  'said',
  'around',
  'news',
  'goals',
  'run',
  'party',
  '26']]

In [26]:
token_vocab([[1,2]])

[['.', ',']]

## 数据集迭代器(Iterator)
神经网络通常是分批训练的。这意味着网络的权值更新是基于每一次的多个序列。每一批中的所有序列需要具有相同的长度。因此，我们将向它们填充一个特殊的< UKN >记号。同样，token和tag也必须填充它。为循环神经网络(Recurrent Neural Network，RNN)提供序列长度是很好的实践，所以它可以跳过对填充部件的计算。我们在这里供批处理函数batches_generator以节省时间。  
  
  批量生成的一个重要概念是打乱(shuffling)。打乱是从数据集中随机抽取样本,对打乱后的数据进行训练是很重要的，因为从同一类抽取的大量结果样本可能导致模型太过于”纯净“。


In [28]:
from deeppavlov.core.data.data_learning_iterator import DataLearningIterator

从加载的数据集中创建数据集迭代器

In [29]:
data_iterator = DataLearningIterator(dataset)

尝试输出：

In [33]:
next(data_iterator.gen_batches(2, shuffle=True))

((['BUDAPEST', '1996-08-23'],
  ['-',
   'Aleix',
   'Vidal-Quadras',
   '-',
   'Catalan',
   'nationalists',
   'are',
   'demanding',
   'my',
   'defenestration']),
 (['B-LOC', 'O'],
  ['O', 'B-PER', 'I-PER', 'O', 'B-MISC', 'O', 'O', 'O', 'O', 'O']))

In [34]:
next(data_iterator.gen_batches(5, shuffle=True))

((['PUT', 'D', '94.00', 'PCT', '0.94', 'DEM', '2.40', 'PCT', '101.40', 'X'],
  ['SOCCER',
   '-',
   'ETHIOPIA',
   'BEAT',
   'UGANDA',
   'ON',
   'PENALTIES',
   'IN',
   'AFRICAN',
   'NATIONS',
   'CUP',
   '.'],
  ['Croatian',
   'lending',
   'rate',
   'falls',
   'to',
   '8.0',
   'vs',
   '9.1',
   'pct',
   '.'],
  ['NOTE', '-', 'Orii', 'Corp', 'makes', 'automation', 'equipment', '.'],
  ['"',
   'Therefore',
   'I',
   'have',
   'some',
   'doubts',
   'about',
   'achieving',
   'some',
   'form',
   'of',
   'reconciliation',
   '.']),
 (['O', 'O', 'O', 'O', 'O', 'B-MISC', 'O', 'O', 'O', 'O'],
  ['O',
   'O',
   'B-LOC',
   'O',
   'B-LOC',
   'O',
   'O',
   'O',
   'B-MISC',
   'I-MISC',
   'I-MISC',
   'O'],
  ['B-MISC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
  ['O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'O'],
  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']))

## 生成掩码
关于生成训练数据的最后一件事。我们需要生成一个二进制掩码，在这个掩码中，token代表1，其他代表是0。  
这个掩码将阻止通过填充来反向传播。  
此类掩码的一个实例:
~~~
[[1, 1, 0, 0, 0],
 [1, 1, 1, 1, 1]]
~~~
代表这些句子：
~~~
 [['The', 'roof'],
  ['This', 'is', 'my', 'domain', '!']]
~~~
掩码长度必须等于批次中句子的最大长度。

In [35]:
from deeppavlov.models.preprocessors.mask import Mask
get_mask = Mask()

In [36]:
get_mask([['Try', 'to', 'get', 'the', 'mask'], ['Check', 'paddings']])

array([[1., 1., 1., 1., 1.],
       [1., 1., 0., 0., 0.]], dtype=float32)

## 建立一个循环神经网络(RNN)